# modules generated in https://chat.openai.com/share/e87f6356-4272-4e28-9d73-a6175aba09c7

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class zReLU(nn.Module):
    def forward(self, input):
        # Create a mask where both real and imaginary parts are positive
        mask = (input.real >= 0) & (input.imag >= 0)
        # Apply the mask
        return input * mask.float()

In [ ]:
class modReLU(nn.Module):
    def __init__(self, bias_init=0.1):
        """
        Initialize the modReLU module.

        Parameters:
            bias_init (float): Initial value for the bias parameter.
        """
        super(modReLU, self).__init__()
        # Initialize the bias as a learnable parameter
        self.bias = nn.Parameter(torch.tensor([bias_init]))

    def forward(self, input):
        """
        Forward pass of the modReLU activation.

        Parameters:
            input (Tensor): Complex input tensor.

        Returns:
            Tensor: Output tensor after applying modReLU activation.
        """
        modulus = input.abs()  # Compute the modulus of the complex input
        phase = input.angle()  # Compute the phase of the complex input

        # Apply the modReLU function: ReLU(modulus + bias) * e^(j*phase)
        # Ensure the bias is broadcasted correctly over the input dimensions
        activated_modulus = torch.relu(modulus + self.bias)
        output = torch.polar(activated_modulus, phase)  # Reconstruct the complex number
        
        return output

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features, out_features, bias_init=0.1):
        super(ResidualBlock, self).__init__()
        self.linear1 = nn.Linear(in_features, out_features)
        self.bn1 = nn.BatchNorm1d(out_features)  # BatchNorm after the first linear layer
        self.modrelu1 = modReLU(bias_init)
        self.linear2 = nn.Linear(out_features, out_features)
        self.bn2 = nn.BatchNorm1d(out_features)  # BatchNorm after the second linear layer
        self.modrelu2 = modReLU(bias_init)
        # Adjust dimensions if necessary, with batch norm
        if in_features != out_features:
            self.match_dimensions = nn.Sequential(
                nn.Linear(in_features, out_features),
                nn.BatchNorm1d(out_features)
            )
        else:
            self.match_dimensions = None

    def forward(self, x):
        residual = x
        out = self.linear1(x)
        out = self.bn1(out)  # Apply batch norm
        out = self.modrelu1(out)
        out = self.linear2(out)
        out = self.bn2(out)  # Apply batch norm
        out = self.modrelu2(out)
        if self.match_dimensions is not None:
            residual = self.match_dimensions(residual)
        out += residual  # Add input to the output
        return out

class CVNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(CVNN, self).__init__()
        self.layers = nn.ModuleList()
        
        previous_layer_size = input_size
        
        for hidden_size in hidden_sizes:
            self.layers.append(ResidualBlock(previous_layer_size, hidden_size))
            previous_layer_size = hidden_size
            
        # Add batch norm before the final linear layer if necessary
        if hidden_sizes:
            self.layers.append(nn.BatchNorm1d(previous_layer_size))
        self.final_linear = nn.Linear(previous_layer_size, output_size)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.final_linear(x)
        return x

# Example usage
input_size = 10  # Example input size
hidden_sizes = [20, 30]  # Example sizes of hidden layers
output_size = 5  # Example output size

model = CVNN(input_size, hidden_sizes, output_size)
print(model)

In [ ]:
class ComplexBatchNorm(nn.Module):
    def __init__(self, num_features):
        super(ComplexBatchNorm, self).__init__()
        self.mag_bn = nn.BatchNorm1d(num_features)
        # Using a placeholder for phase normalization; consider circular statistics for actual implementation
        self.phase_bn = nn.BatchNorm1d(num_features)

    def forward(self, x):
        magnitude = torch.abs(x)
        phase = torch.angle(x)

        normalized_mag = self.mag_bn(magnitude)
        normalized_phase = self.phase_bn(phase)  # Placeholder

        # Reconstruct complex numbers from normalized magnitude and phase
        x_normalized = torch.polar(normalized_mag, normalized_phase)
        return x_normalized

class modReLU(nn.Module):
    def __init__(self, bias_init=0.1):
        super(modReLU, self).__init__()
        self.bias = nn.Parameter(torch.tensor([bias_init]))

    def forward(self, input):
        # Adapted for complex inputs
        magnitude = torch.abs(input)
        activated_modulus = F.relu(magnitude + self.bias)
        phase = torch.angle(input)
        return torch.polar(activated_modulus, phase)

class ResidualBlock(nn.Module):
    def __init__(self, in_features, out_features, bias_init=0.1):
        super(ResidualBlock, self).__init__()
        self.linear1 = nn.Linear(in_features, out_features)
        self.bn1 = ComplexBatchNorm(out_features)  # Complex BatchNorm
        self.modrelu1 = modReLU(bias_init)
        self.linear2 = nn.Linear(out_features, out_features)
        self.bn2 = ComplexBatchNorm(out_features)  # Complex BatchNorm
        self.modrelu2 = modReLU(bias_init)

        if in_features != out_features:
            self.match_dimensions = nn.Sequential(
                nn.Linear(in_features, out_features),
                ComplexBatchNorm(out_features)  # Complex BatchNorm for matching dimensions
            )
        else:
            self.match_dimensions = None

    def forward(self, x):
        residual = x
        out = self.linear1(x)
        out = self.bn1(out)  # Apply complex batch norm
        out = self.modrelu1(out)
        out = self.linear2(out)
        out = self.bn2(out)  # Apply complex batch norm
        out = self.modrelu2(out)
        if self.match_dimensions is not None:
            residual = self.match_dimensions(residual)
        out += residual  # Add input to the output
        return out

class CVNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(CVNN, self).__init__()
        self.layers = nn.ModuleList()

        previous_layer_size = input_size

        for hidden_size in hidden_sizes:
            self.layers.append(ResidualBlock(previous_layer_size, hidden_size))
            previous_layer_size = hidden_size

        self.final_linear = nn.Linear(previous_layer_size, output_size)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.final_linear(x)
        return x


In [ ]:
class CircularStatistics(nn.Module):
    def __init__(self, num_features, eps=1e-5):
        super(CircularStatistics, self).__init__()
        self.eps = eps
        # Learnable parameters for circular data
        self.gamma = nn.Parameter(torch.ones(num_features))
        self.beta = nn.Parameter(torch.zeros(num_features))

    def forward(self, phase):
        # Convert phase to complex representation
        complex_phase = torch.exp(1j * phase)

        # Circular mean
        circular_mean = torch.atan2(complex_phase.imag.mean(dim=0), complex_phase.real.mean(dim=0))

        # Circular variance (using length of mean resultant vector)
        norm = torch.abs(complex_phase.mean(dim=0))
        circular_variance = 1 - norm

        # Normalize phase
        phase_normalized = phase - circular_mean.unsqueeze(0)
        phase_normalized = torch.atan2(torch.sin(phase_normalized), torch.cos(phase_normalized))  # Wrap-around effect

        # Apply learnable parameters
        phase_normalized = self.gamma * (phase_normalized / (circular_variance.sqrt().unsqueeze(0) + self.eps)) + self.beta

        return phase_normalized

class ComplexBatchNorm(nn.Module):
    def __init__(self, num_features):
        super(ComplexBatchNorm, self).__init__()
        self.mag_bn = nn.BatchNorm1d(num_features)  # BatchNorm for magnitude
        self.phase_bn = CircularStatistics(num_features)  # Circular statistics for phase

    def forward(self, x):
        magnitude = torch.abs(x)
        phase = torch.angle(x)

        # Normalize magnitude with standard BatchNorm
        normalized_mag = self.mag_bn(magnitude)
        
        # Normalize phase with CircularStatistics
        normalized_phase = self.phase_bn(phase)

        # Reconstruct complex numbers
        x_normalized = torch.polar(normalized_mag, normalized_phase)
        return x_normalized

In [ ]:
# suggested baseline
class SimpleCVNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(SimpleCVNN, self).__init__()
        self.layers = nn.ModuleList()
        
        # Starting layer sizes configuration
        previous_layer_size = input_size
        
        # Constructing the hidden layers
        for hidden_size in hidden_sizes:
            self.layers.append(nn.Linear(previous_layer_size, hidden_size))
            self.layers.append(nn.ReLU())  # Using ReLU for simplicity; replace with a complex-compatible activation if necessary
            previous_layer_size = hidden_size
        
        # Final layer to produce the output
        self.final_linear = nn.Linear(previous_layer_size, output_size)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)  # Apply each layer in sequence
        x = self.final_linear(x)
        return x

# Example model instantiation
input_size = 100  # Number of input features
hidden_sizes = [128, 64]  # Sizes of hidden layers
output_size = 10  # Number of output features

model = SimpleCVNN(input_size, hidden_sizes, output_size)

print(model)
